<a href="https://colab.research.google.com/github/ancestor9/2025_capstone/blob/main/%EA%B0%95%EC%9D%98%EC%8B%A4%EB%B0%B0%EC%A0%95_%EC%A0%95%EC%88%98%EA%B3%84%ED%9A%8D%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp -q

## 강의 시간표 배정 문제의 정수계획법 모델

제공된 코드는 강의, 시간대, 강의실에 대한 제약 조건을 만족하는 실행 가능한 시간표를 찾는 **실행 가능성 문제(Feasibility Problem)**를 모델링합니다.

---
## 1. 집합 (Sets)

- $C$: 모든 교과목의 집합 ($i \in C$)  
- $T$: 모든 시간대의 집합 ($t \in T$)  
- $R$: 모든 강의실의 집합 ($r \in R$)  
- $P$: 모든 교수의 집합 ($p \in P$)  
- $C_p$: 특정 교수 $p$가 담당하는 교과목들의 집합 ($C_p \subseteq C$)  


---
## 2. 변수 및 매개변수 (Variables and Parameters)

### 결정 변수 (Decision Variable)

교과목 $i$가 시간대 $t$에 강의실 $r$에 배정되었는지를 나타내는 변수:

$$
x_{i,t,r} =
\begin{cases}
1 & \text{교과목 $i$가 시간대 $t$에 강의실 $r$에 배정된 경우} \\
0 & \text{그 외의 경우}
\end{cases}
$$

$$
x_{i,t,r} \in \{0,1\}, \quad \forall i \in C, \forall t \in T, \forall r \in R
$$

---

### 매개변수 (Parameters)

- $h_i$: 교과목 $i$의 필수 시수


---

## 3. 목적식 (Objective Function)

본 모델은 최적화가 아닌 실행 가능성 문제이므로, 단순히 제약조건을 만족하는 해를 찾는 것이 목표입니다.  
따라서 목적식은 다음과 같이 표현됩니다.

$$
\text{Minimize } Z = 0
$$

---

## 4. 제약 조건 (Constraints)

### 제약 조건 1: 시수 제약
각 교과목은 정해진 시수만큼 정확히 배정되어야 한다.  

$$
\sum_{t \in T} \sum_{r \in R} x_{i,t,r} = h_i, \quad \forall i \in C
$$

---

### 제약 조건 2: 강의실 동시 사용 제약
특정 시간대에 한 강의실은 오직 하나의 교과목에만 사용될 수 있다.  

$$
\sum_{i \in C} x_{i,t,r} \leq 1, \quad \forall t \in T, \forall r \in R
$$

---

### 제약 조건 3: 교수 동시 강의 제약
각 교수는 특정 시간대에 하나의 교과목만 강의할 수 있다.  

$$
\sum_{i \in C_p} \sum_{r \in R} x_{i,t,r} \leq 1, \quad \forall t \in T, \forall p \in P
$$

---

### 제약 조건 4: 교과목 동시 배정 제약
각 교과목은 특정 시간대에 하나의 강의실에만 배정될 수 있다.  

$$
\sum_{r \in R} x_{i,t,r} \leq 1, \quad \forall i \in C, \forall t \in T
$$


In [17]:
import pulp
import numpy as np
from itertools import product

def create_courses():
    """테스트용 교과목 데이터"""
    courses = [
        {"name": "알고리즘", "hours": 3, "professor": "김민준", "type": "컴퓨터", "grade": 3, "students": 25},
        {"name": "데이터베이스", "hours": 2, "professor": "이서연", "type": "일반", "grade": 2, "students": 35},
        {"name": "운영체제", "hours": 3, "professor": "최유진", "type": "컴퓨터", "grade": 3, "students": 28},
        {"name": "컴퓨터네트워크", "hours": 2, "professor": "정하윤", "type": "일반", "grade": 3, "students": 40},
        {"name": "머신러닝", "hours": 3, "professor": "박지훈", "type": "컴퓨터", "grade": 4, "students": 20},
        {"name": "알고리즘1", "hours": 3, "professor": "김민준", "type": "컴퓨터", "grade": 3, "students": 25},
        {"name": "데이터베이스1", "hours": 2, "professor": "이서연", "type": "일반", "grade": 2, "students": 35},
        {"name": "운영체제1", "hours": 3, "professor": "최유진", "type": "컴퓨터", "grade": 3, "students": 28},
        {"name": "컴퓨터네트워크1", "hours": 2, "professor": "정하윤", "type": "일반", "grade": 3, "students": 40},
        {"name": "머신러닝1", "hours": 3, "professor": "박지훈", "type": "컴퓨터", "grade": 4, "students": 20},
        {"name": "알고리즘2", "hours": 6, "professor": "조상1", "type": "컴퓨터", "grade": 3, "students": 25},
    ]
    return courses

# ✅ classrooms를 딕셔너리로 정의
classrooms = {
    "1215": {"type": "컴퓨터", "capacity": 30},
    "1216": {"type": "일반", "capacity": 40},
    "1217": {"type": "컴퓨터", "capacity": 30},
    "1416": {"type": "일반", "capacity": 50},
    "1218": {"type": "컴퓨터", "capacity": 30}
}

# 기본 설정
courses = create_courses()
time_slots = [f"{h:02d}:00" for h in range(9, 18)]  # 09:00-17:00


### 과목과 강의실 배정

In [20]:
def solve_timetable():
    """연속시간 배정을 고려한 강의 시간표 배정"""

    print("연속시간 배정 강의 시간표 최적화 시작...")
    print("=" * 50)

    try:
        # 문제 정의
        prob = pulp.LpProblem("timetable", pulp.LpMinimize)

        # 결정 변수: y[i,r,t] = 강의 i가 강의실 r에서 시간 t부터 시작하면 1
        y = {}
        valid_assignments = []

        for i, course in enumerate(courses):
            for r, (room_name, room_info) in enumerate(classrooms.items()):
                # 타입과 수용인원 체크
                if course['type'] == room_info['type'] and course['students'] <= room_info['capacity']:
                    # 연속 시간을 고려한 시작 가능 시간
                    max_start = len(time_slots) - course['hours']
                    for t in range(max_start + 1):
                        var_name = f"y_{i}_{r}_{t}"
                        y[(i, r, t)] = pulp.LpVariable(var_name, cat='Binary')
                        valid_assignments.append((i, r, t))

        print(f"생성된 변수 수: {len(y)}")

        # 목적함수 (더미)
        prob += 0

        # 제약조건 1: 각 강의는 정확히 한 번만 배정
        for i, course in enumerate(courses):
            assignments = [y[(i, r, t)] for (i2, r, t) in valid_assignments if i2 == i]
            if assignments:
                prob += pulp.lpSum(assignments) == 1

        # 제약조건 2: 강의실 시간 충돌 방지
        for r in range(len(classrooms)):
            for t in range(len(time_slots)):
                conflicting = []
                for (i, r2, start_t) in valid_assignments:
                    if r2 == r:
                        course = courses[i]
                        # 시간 t가 이 강의의 시간 범위에 포함되는지 확인
                        if start_t <= t < start_t + course['hours']:
                            conflicting.append(y[(i, r2, start_t)])

                if conflicting:
                    prob += pulp.lpSum(conflicting) <= 1

        # 제약조건 3: 교수 시간 충돌 방지
        professors = {}
        for i, course in enumerate(courses):
            prof = course['professor']
            if prof not in professors:
                professors[prof] = []
            professors[prof].append(i)

        for prof, course_ids in professors.items():
            if len(course_ids) > 1:
                for t in range(len(time_slots)):
                    conflicting = []
                    for i in course_ids:
                        course = courses[i]
                        for (i2, r, start_t) in valid_assignments:
                            if i2 == i and start_t <= t < start_t + course['hours']:
                                conflicting.append(y[(i2, r, start_t)])

                    if conflicting:
                        prob += pulp.lpSum(conflicting) <= 1

        print(f"제약조건 설정 완료")

        # 해결
        prob.solve()

        # 상태 확인 (호환성 개선)
        status_code = prob.status
        if status_code == 1:  # Optimal
            status = "Optimal"
            return prob, y, valid_assignments, status
        else:
            status = f"Status_{status_code}"
            return prob, None, None, status

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None, None, None, f"Error: {str(e)}"

def display_results(prob, y, valid_assignments, status):
    """결과 출력"""

    print("\n" + "="*50)
    print("연속시간 배정 시간표 결과")
    print("="*50)

    if y is None or status != "Optimal":
        print(f"해결책을 찾을 수 없습니다. 상태: {status}")
        return

    # 해결책 추출
    solution = {}
    for (i, r, t) in valid_assignments:
        if y[(i, r, t)].varValue == 1:
            course = courses[i]
            room_name = list(classrooms.keys())[r]
            time_range = [time_slots[t + h] for h in range(course['hours'])]

            solution[course['name']] = {
                'classroom': room_name,
                'start_time': t,
                'time_slots': time_range,
                'professor': course['professor'],
                'students': course['students'],
                'hours': course['hours']
            }

    # 결과 출력
    print(f"\n배정된 강의 수: {len(solution)}/{len(courses)}")
    print("-" * 40)

    for course_name, info in solution.items():
        room_info = classrooms[info['classroom']]
        room_type = "컴퓨터실" if room_info['type'] == "컴퓨터" else "일반강의실"

        print(f"\n{course_name} ({info['professor']})")
        print(f"  시간: {' → '.join(info['time_slots'])}")
        print(f"  강의실: {info['classroom']} ({room_type}, {room_info['capacity']}명)")
        print(f"  수강생: {info['students']}명")

    # 강의실별 시간표
    print(f"\n\n강의실별 시간표:")
    print("="*40)

    for room_name, room_info in classrooms.items():
        print(f"\n{room_name} ({room_info['type']}, {room_info['capacity']}명)")
        print("-" * 30)

        room_schedule = {}
        for course_name, info in solution.items():
            if info['classroom'] == room_name:
                for time_slot in info['time_slots']:
                    room_schedule[time_slot] = f"{course_name} ({info['professor']})"

        for time_slot in time_slots:
            if time_slot in room_schedule:
                print(f"  {time_slot}: {room_schedule[time_slot]}")
            else:
                print(f"  {time_slot}: 비어있음")

    # 연속성 검증
    print(f"\n\n연속성 검증:")
    print("-" * 20)
    all_consecutive = True

    for course_name, info in solution.items():
        expected_hours = info['hours']
        actual_hours = len(info['time_slots'])

        # 시간 연속성 확인
        time_indices = [time_slots.index(ts) for ts in info['time_slots']]
        is_consecutive = all(time_indices[i] + 1 == time_indices[i + 1]
                           for i in range(len(time_indices) - 1))

        status_text = "연속" if is_consecutive else "비연속"
        print(f"{course_name}: {actual_hours}시간, {status_text}")

        if not is_consecutive:
            all_consecutive = False

    print(f"\n전체 연속성: {'성공' if all_consecutive else '실패'}")

def main():
    """메인 실행 함수"""
    print("연속시간 배정 강의 시간표 자동 배정 시스템")
    print("="*50)

    print(f"교과목 수: {len(courses)}")
    print(f"강의실 수: {len(classrooms)}")
    print(f"시간대: {len(time_slots)}시간")

    try:
        prob, y, valid_assignments, status = solve_timetable()
        display_results(prob, y, valid_assignments, status)

    except Exception as e:
        print(f"실행 중 오류: {str(e)}")
        print("PuLP가 올바르게 설치되어 있는지 확인하세요:")
        print("pip install pulp")

if __name__ == "__main__":
    main()

연속시간 배정 강의 시간표 자동 배정 시스템
교과목 수: 11
강의실 수: 5
시간대: 9시간
연속시간 배정 강의 시간표 최적화 시작...
생성된 변수 수: 202
제약조건 설정 완료

연속시간 배정 시간표 결과

배정된 강의 수: 11/11
----------------------------------------

알고리즘 (김민준)
  시간: 15:00 → 16:00 → 17:00
  강의실: 1217 (컴퓨터실, 30명)
  수강생: 25명

데이터베이스 (이서연)
  시간: 09:00 → 10:00
  강의실: 1216 (일반강의실, 40명)
  수강생: 35명

운영체제 (최유진)
  시간: 09:00 → 10:00 → 11:00
  강의실: 1215 (컴퓨터실, 30명)
  수강생: 28명

컴퓨터네트워크 (정하윤)
  시간: 11:00 → 12:00
  강의실: 1216 (일반강의실, 40명)
  수강생: 40명

머신러닝 (박지훈)
  시간: 15:00 → 16:00 → 17:00
  강의실: 1215 (컴퓨터실, 30명)
  수강생: 20명

알고리즘1 (김민준)
  시간: 12:00 → 13:00 → 14:00
  강의실: 1215 (컴퓨터실, 30명)
  수강생: 25명

데이터베이스1 (이서연)
  시간: 13:00 → 14:00
  강의실: 1216 (일반강의실, 40명)
  수강생: 35명

운영체제1 (최유진)
  시간: 12:00 → 13:00 → 14:00
  강의실: 1217 (컴퓨터실, 30명)
  수강생: 28명

컴퓨터네트워크1 (정하윤)
  시간: 15:00 → 16:00
  강의실: 1216 (일반강의실, 40명)
  수강생: 40명

머신러닝1 (박지훈)
  시간: 09:00 → 10:00 → 11:00
  강의실: 1217 (컴퓨터실, 30명)
  수강생: 20명

알고리즘2 (조상1)
  시간: 09:00 → 10:00 → 11:00 → 12:00 → 13:00 → 14:00
  강의실: 1218 (컴퓨터실, 30